In [1]:
import cv2 as cv
import numpy as np
import pandas as pd
import csv

In [2]:
def get_punctuation(person_id, video_id):
    df = pd.read_csv("data/facecut/input/scores.csv", index_col = False)
    
    filtered = df.loc[df["idPersona"] == person_id]

    return filtered.loc[df["idVideo"] == video_id]["puntuacion"].to_string(index = False)

In [3]:
def write_row(person_id, video_id, nframe, face, punctuation):
    face_string = ""
    for i in face:
        face_string += str(i) + " "
    path = "data/facecut/output_csv/faces.csv"
    with open(path, 'a') as f:
        writer = csv.writer(f, lineterminator='\n')
        writer.writerow([person_id, video_id, nframe, face_string, punctuation])

In [4]:
def detect_faces(route):
    original_image = route
    grayscale_image = cv.cvtColor(original_image, cv.COLOR_BGR2GRAY)
    grayscale_image = original_image
    face_cascade = cv.CascadeClassifier('data/face_classifier.xml')

    detected_faces = face_cascade.detectMultiScale(grayscale_image)

    for (column, row, width, height) in detected_faces:
        cv.rectangle(
            original_image,
            (column, row),
            (column + width, row + height),
            (0, 255, 0),
            2
        )

    return detected_faces

In [5]:
def cut_face(img, face_coords):
    x0 = face_coords[0]
    y0 = face_coords[1]
    
    width = face_coords[2]
    height = face_coords[3]
    
    x1 = x0 + width
    y1 = y0 + height

    
    return img[y0:y1, x0:x1]

In [6]:
def face2array(frame, faces_detected):
    only_face = cut_face(frame, faces_detected[0])
    small_face = cv.resize(only_face, (48, 48))
    grey_face = cv.cvtColor(small_face, cv.COLOR_BGR2GRAY)
    flat_array = np.array(grey_face).flatten()
    
    return flat_array, grey_face

In [7]:
def manage_face(frame, faces_detected, person_id, video_id, nframe):
    flat_array, grey_face = face2array(frame, faces_detected)
    punctuation = get_punctuation(person_id, video_id)
    write_row(person_id, video_id, int(nframe/10), flat_array, punctuation)
    cv.imwrite("data/facecut/output_frames/" + str(person_id) + "-" + str(video_id) + "-"+ str(int(nframe/10)) + ".jpg", grey_face)

In [8]:
def main():    
    person_id = "s04"
    video_len = 20

    for video_id in range(1, video_len):
        video = cv.VideoCapture("data/facecut/input/video" + str(video_id) + "-" + person_id + ".mp4")
        print("Cutting video:  " + str(video_id))
        nframe = 0

        while True:
            check, frame = video.read()

            if(not check):
                 break

            faces_detected = detect_faces(frame)
            cv.imshow("Vital Emo - Cutting Faces", frame)

            if type(faces_detected) is not tuple and faces_detected.shape[0] == 1:
                if nframe % 10 == 0:
                    manage_face(frame, faces_detected, person_id, video_id, nframe)
                nframe += 1
            key = cv.waitKey(1)

            if key == ord('q'):
                break

    video.release()
    cv.destroyAllWindows()

In [9]:
main()

Cutting video:  1
Cutting video:  2
Cutting video:  3
Cutting video:  4
Cutting video:  5
Cutting video:  6
Cutting video:  7
Cutting video:  8
Cutting video:  9
Cutting video:  10
Cutting video:  11
Cutting video:  12
Cutting video:  13
Cutting video:  14
Cutting video:  15
Cutting video:  16
Cutting video:  17
Cutting video:  18
Cutting video:  19
